#Call GIT, INSTALL PACKAGES & IMPORT PACKAGES

In [ ]:
#!git clone https://github.com/zhouhaoyi/ETDataset.git
#!git clone https://github.com/RL-NAS-TSF4E/RL-NAS-TSF4E.git

#import sys
#if not '/content/RL-NAS-TSF4E' in sys.path:
#    sys.path += ['/content/RL-NAS-TSF4E']

In [1]:
import sys
if not '/vol/fob-vol3/mi20/jaabousa/Master_thesis_experiments/RL-NAS-TSF4E' in sys.path:
    sys.path += ['/vol/fob-vol3/mi20/jaabousa/Master_thesis_experiments/RL-NAS-TSF4E']

In [2]:
#!pip install stable-baselines3
#!pip install 'shimmy>=0.2.1'
#!pip install gym
#!pip install optuna

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import os
import pickle
import time
from datetime import datetime

import torch.optim as optim

import utils
import models

#from data_loader import Dataset_ETT_hour
#from data_loader_factory import get_data
import data_factory
from model_trainer import _process_one_batch, vali, train, _evaluate

from informerStudy import InformerStudy

import rl_nas

#from rl_nas.environment.rl_early_termination import StopTrainingAfterNStepsCallback
#from rl_nas.environment.rl_nas_env import RLNASEnv

#from rl_nas.agents.a2c_agent import A2CAgent
#from rl_nas.agents.ppo_agent import PPOAgent

#from rl_nas_env import RLNASEnv
#from rl_early_termination import StopTrainingAfterNStepsCallback
#from a2c_agent import A2CAgent
#from ppo_agent import PPOAgent

#import matplotlib.pyplot as plt
#import seaborn as sns

import optuna

import gym
import shimmy
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

/vol/fob-vol3/mi20/jaabousa/Dokumente/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#Initiate Experiment Parameters

In [4]:
use_gpu = True if torch.cuda.is_available() else False
gpu = 1
use_multi_gpu = True
devices = '0,1,2,3'

def _acquire_device():
        if use_gpu:
            os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu) if not use_multi_gpu else devices
            device = torch.device('cuda:{}'.format(gpu))
            #print('Use GPU: cuda:{}'.format(gpu))
        else:
            device = torch.device('cpu')
            #print('Use CPU')
        return device

device=_acquire_device()

# model&datset
seq_len = 96
label_len = 48
pred_len = 24

e_layers=2
d_layers=1
dropout=0.05

embed='timeF'
freq='h'

#datasets
features='M'
target='OT'
inverse=False

#data_loader
num_workers = 0
batch_size = 32

path = './informer_checkpoints'
if not os.path.exists(path):
    os.makedirs(path)

padding = 0
lradj = 'type1'
train_epochs = 6

#Default Model

In [5]:
#build default model
model = models.Informer(
    enc_in=7,
    dec_in=7,
    c_out=7,
    seq_len=seq_len,
    label_len=label_len,
    out_len=pred_len,
    e_layers=e_layers,
    d_layers=d_layers,
    factor=5,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    dropout=dropout,
    attn='prob',
    embed=embed,
    freq=freq,
    activation='gelu',
    output_attention=False,
    distil=True,
    mix=True,
).float()

model = model.to(device)

#load data
train_data, train_loader = data_factory.get_data('train')
test_data, test_loader = data_factory.get_data('test')
val_data, val_loader = data_factory.get_data('val')

#initiate parameters
learning_rate = 0.0001

patience=3

train_steps = len(train_loader)
early_stopping = utils.EarlyStopping(patience=patience, verbose=True)

model_optim = optim.Adam(model.parameters(), lr=learning_rate)
criterion =  nn.MSELoss()

#train model
train(model, train_loader, val_data, val_loader, criterion, model_optim, path, train_epochs, learning_rate, lradj, early_stopping, device, padding, pred_len, label_len , inverse, features)

#evaluate on validation data
default_preds, default_trues, default_val_mae, default_val_mse, default_val_rmse, default_val_mape, default_val_mspe = _evaluate(model, val_loader, val_data, _process_one_batch, device, padding, pred_len, label_len, inverse, features)

preds, trues, test_mae, test_mse, test_rmse, test_mape, test_mspe = _evaluate(model, test_loader, test_data, _process_one_batch, device, padding, pred_len, label_len, inverse, features)

torch.save(model.state_dict(), path+'/'+'default_informer.pth')

# result save
folder_path = './results/default/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(folder_path+'metrics.npy', np.array([test_mae, test_mse, test_rmse, test_mape, test_mspe]))
np.save(folder_path+'pred.npy', preds)
np.save(folder_path+'true.npy', trues)

torch.cuda.empty_cache()

#Optuna

In [6]:
informer_study = InformerStudy(seq_len, label_len, pred_len, embed, freq, device, path, train_epochs, lradj, padding, inverse, features, train_loader, val_data, val_loader, test_data, test_loader)
study, op_preds, op_trues, op_test_mae, op_test_mse, op_test_rmse, op_test_mape, op_test_mspe, op_actions_mae = informer_study.run_study(n_trials=500)

# result save
folder_path = './results/optuna/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(folder_path+'optuna_metrics.npy', np.array([op_test_mae, op_test_mse, op_test_rmse, op_test_mape, op_test_mspe]))
np.save(folder_path+'optuna_pred.npy', op_preds)
np.save(folder_path+'optuna_true.npy', op_trues)
with open(folder_path + 'optuna_actions_mae.pkl', 'wb') as f:
    pickle.dump(op_actions_mae, f)

torch.cuda.empty_cache()

[I 2024-05-10 18:44:19,708] A new study created in memory with name: no-name-3ff571fb-17fa-4910-b20c-c37786e9ccac


[I 2024-05-10 18:47:10,549] Trial 0 finished with value: 0.6109031438827515 and parameters: {'e_layers': 3, 'd_layers': 4, 'dropout': 0.15, 'factor': 7, 'n_heads': 5, 'd_ff': 512, 'd_model': 1024}. Best is trial 0 with value: 0.6109031438827515.
[I 2024-05-10 18:48:40,232] Trial 1 finished with value: 0.5566806197166443 and parameters: {'e_layers': 1, 'd_layers': 3, 'dropout': 0.15, 'factor': 5, 'n_heads': 7, 'd_ff': 512, 'd_model': 512}. Best is trial 1 with value: 0.5566806197166443.
[I 2024-05-10 18:50:54,679] Trial 2 finished with value: 0.6184190511703491 and parameters: {'e_layers': 5, 'd_layers': 2, 'dropout': 0.05, 'factor': 4, 'n_heads': 8, 'd_ff': 1024, 'd_model': 1024}. Best is trial 1 with value: 0.5566806197166443.
[I 2024-05-10 18:52:24,342] Trial 3 finished with value: 0.5854873657226562 and parameters: {'e_layers': 2, 'd_layers': 4, 'dropout': 0.15, 'factor': 7, 'n_heads': 7, 'd_ff': 1024, 'd_model': 512}. Best is trial 1 with value: 0.5566806197166443.
[I 2024-05-10 18

#RL-NAS PPO

In [7]:
env = rl_nas.environment.RLNASEnv(seq_len, label_len, pred_len, embed, freq, device, path, train_epochs, lradj, padding, inverse, features, train_loader, val_data, val_loader, default_val_mae)

agent = rl_nas.agents.PPOAgent(seq_len, label_len, pred_len, embed, freq, device, path, train_epochs, lradj, padding, inverse, features, train_loader, val_data, val_loader, env)

ppo_actions_mae, best_parameters = agent.get_ppo_results(steps=500)
ppo_preds, ppo_trues, ppo_mae, ppo_mse, ppo_rmse, ppo_mape, ppo_mspe = agent.get_best_model(best_parameters)

# result save
folder_path = './results/ppo/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(folder_path+'ppo_metrics.npy', np.array([ppo_test_mae, ppo_test_mse, ppo_test_rmse, ppo_test_mape, ppo_test_mspe]))
np.save(folder_path+'ppo_pred.npy', ppo_preds)
np.save(folder_path+'ppo_true.npy', ppo_trues)
with open(folder_path + 'ppo_actions_mae.pkl', 'wb') as f:
    pickle.dump(ppo_actions_mae, f)

torch.cuda.empty_cache()

/vol/fob-vol3/mi20/jaabousa/Dokumente/.conda/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
[I 2024-05-11 07:20:51.007223] Trial 0 finished with value: 0.5686572790145874 and parameters: 'e_layers': 2 , 'd_layers': 4, 'dropout': 0.1, 'factor': 6, 'n_heads': 5, ' d_ff: ' 2048, 'd_model: '512. Best is trial default with value: 0.5597437024116516.
[I 2024-05-11 07:23:50.278562] Trial 1 finished with value: 0.6139858961105347 and parameters: 'e_layers': 6 , 'd_layers': 3, 'dropout': 0.15, 'factor': 8, 'n_heads': 8, ' d_ff: ' 512, 'd_model: '512. Best is trial default with value: 0.5597437024116516.
[I 2024-05-11 07:26:02.779968] Trial 2 finished with value: 0.6211072206497192 and parameters: 'e_layers': 5 , 'd_layers': 2, 'dropout': 0.15, 'factor': 5, 'n_heads': 8, ' d_ff: ' 512, 'd_model: '1024. Best is trial default with value: 0.5597437024116516.
[I 2024-05-11 07:29:51.649670] Trial 3 finished with value: 0.6298952102661133 and parameters: 'e_layers': 4 , 'd_layers': 6, 'dropout': 0.1, 'factor': 4, 'n_heads': 4, ' d_ff: ' 2048, 'd_model: '1024. Best is trial 

#RL-NAS A2C

In [8]:
env = rl_nas.environment.RLNASEnv(seq_len, label_len, pred_len, embed, freq, device, path, train_epochs, lradj, padding, inverse, features, train_loader, val_data, val_loader, default_val_mae)

agent = rl_nas.agents.A2CAgent(seq_len, label_len, pred_len, embed, freq, device, path, train_epochs, lradj, padding, inverse, features, train_loader, val_data, val_loader, env)

a2c_actions_mae, best_parameters = agent.get_a2c_results(steps=500)
a2c_preds, a2c_trues, a2c_mae, a2c_mse, a2c_rmse, a2c_mape, a2c_mspe = agent.get_best_model(best_parameters)

# result save
folder_path = './results/a2c/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(folder_path+'a2c_metrics.npy', np.array([a2c_mae, a2c_mse, a2c_rmse, a2c_mape, a2c_mspe]))
np.save(folder_path+'a2c_pred.npy', a2c_preds)
np.save(folder_path+'a2c_true.npy', a2c_trues)
with open(folder_path + 'a2c_actions_mae.pkl', 'wb') as f:
    pickle.dump(a2c_actions_mae, f)

torch.cuda.empty_cache()

Using cuda device
[I 2024-05-12 10:45:02.233019] Trial 0 finished with value: 0.6048898696899414 and parameters: 'e_layers': 3 , 'd_layers': 3, 'dropout': 0.05, 'factor': 5, 'n_heads': 7, ' d_ff: ' 1024, 'd_model: '1024. Best is trial default with value: 0.5597437024116516.
[I 2024-05-12 10:47:43.412150] Trial 1 finished with value: 0.6048887968063354 and parameters: 'e_layers': 5 , 'd_layers': 3, 'dropout': 0.0, 'factor': 6, 'n_heads': 4, ' d_ff: ' 2048, 'd_model: '512. Best is trial default with value: 0.5597437024116516.
[I 2024-05-12 10:50:24.860794] Trial 2 finished with value: 0.5950373411178589 and parameters: 'e_layers': 3 , 'd_layers': 3, 'dropout': 0.05, 'factor': 8, 'n_heads': 5, ' d_ff: ' 1024, 'd_model: '1024. Best is trial default with value: 0.5597437024116516.
[I 2024-05-12 10:51:40.355130] Trial 3 finished with value: 0.5766992568969727 and parameters: 'e_layers': 2 , 'd_layers': 1, 'dropout': 0.05, 'factor': 6, 'n_heads': 6, ' d_ff: ' 2048, 'd_model: '512. Best is tri